In [2]:
# Import libraries and files
import numpy as np
import pandas as pd
import glob

acne_dataset = 'datasets/acne-severity/'
train_data = acne_dataset + 'train.txt'
test_data = acne_dataset + 'test.txt'

path = acne_dataset + 'JPEGImages'


In [3]:
from PIL import Image
import os
import torch
from sklearn.model_selection import train_test_split

# Define class
class Classification_Dataset:
    def __init__(self, data, data_path, transform, training=True):
        self.data = data
        self.imgs = data['path'].unique().tolist()
        self.data_path = data_path
        self.training = training
        self.transform = transform
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.data_path, self.data.iloc[idx, 0]))

        if(self.training):
            label = self.data.iloc[idx, 1]
        if self.transform is not None:
            img = self.transform(img)
        if(self.training):
            return img, label
        else:
            return img

    def __len__(self):
        return len(self.imgs)
        
        
def make_loader(dataset, train_batch_size, validation_split=0.2):
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    print(len(train_set))
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=train_batch_size, shuffle=True,
    )
    # create test_loader
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_loader, test_loader


def data_split(data, test_size):
    x_train, x_test, y_train, y_test = train_test_split(
        data, data["label"], test_size=test_size, stratify = data.iloc[:,1]
    )
    return x_train, x_test, y_train, y_test

In [4]:
from sklearn import metrics as skmetrics
import numpy
class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [5]:
import torch.nn.functional as F
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.dim = dim

    def forward(self, pred, target):
        target = F.one_hot(target, num_classes=pred.size(-1))
        target = target.float()
        target = (1 - self.smoothing) * target + self.smoothing / pred.size(-1)
        log_pred = F.log_softmax(pred, dim=self.dim)
        loss = nn.KLDivLoss(reduction='batchmean')(log_pred, target)
        return loss
    
criterion = LabelSmoothingLoss(smoothing=0.12)

In [6]:

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
     
        self.cnn = torchvision.models.efficientnet_v2_m(pretrained=True).cuda()
        for param in self.cnn.parameters():
            param.requires_grad = True
        self.cnn.classifier = nn.Sequential(
      
        nn.Linear(self.cnn.classifier[1].in_features, 512),
        nn.Dropout(p=0.2),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.Dropout(p=0.2),
        nn.Linear(128, 64),
            nn.Linear(64, 4),     
       )
        
    def forward(self, img):
        output = self.cnn(img)
        return output

In [7]:
def train_one_epoch(
    model,
    train_loader,
    test_loader,
    device,
    optimizer,
    criterion,
    train_metrics,
    val_metrics,
):

    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor).to(device)
#         day = day.view(-1,1).type(torch.FloatTensor).to(device)
        # target=torch.Tensor(target)
        target = target.float().to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    train_metrics.step(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor).to(device)
#             day = day.view(-1,1).type(torch.FloatTensor).to(device)
            target = target.to(device)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_metrics.step(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        train_metrics.last_step_metrics(),
        val_metrics.last_step_metrics(),
    )

In [8]:
BATCH_SIZE = 32

In [9]:
train_df = pd.read_csv(train_data ,names=['path','label','leisons'],sep='  ', engine='python')
train_df

,path,label,leisons
0,levle0_0.jpg,0,3
1,levle0_1.jpg,0,3
2,levle0_100.jpg,0,2
3,levle0_101.jpg,0,1
4,levle0_102.jpg,0,3
...,...,...,...
1160,levle3_92.jpg,3,63
1161,levle3_93.jpg,3,61
1162,levle3_94.jpg,3,58
1163,levle3_95.jpg,3,58


In [10]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [11]:
test_df = pd.read_csv(test_data, names=['path','label','leisons'],sep='  ', engine='python')
test_df

,path,label,leisons
0,levle0_451.jpg,0,2
1,levle0_498.jpg,0,1
2,levle0_485.jpg,0,3
3,levle0_218.jpg,0,2
4,levle1_344.jpg,0,4
...,...,...,...
287,levle3_88.jpg,3,64
288,levle3_1.jpg,3,60
289,levle3_68.jpg,3,56
290,levle3_16.jpg,3,65


In [12]:
import torchvision
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
                                            torchvision.transforms.RandomVerticalFlip(p=0.5),
                                            torchvision.transforms.RandomRotation(degrees=15),
                                            torchvision.transforms.ElasticTransform(),
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(mean, std)])

test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize(mean, std)])

In [13]:
train_dataset = Classification_Dataset(x_train, 
                                        data_path = acne_dataset + "JPEGImages", 
                                        transform=transform, 
                                        training=True)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=BATCH_SIZE, 
                                            shuffle=True,
)

val_dataset = Classification_Dataset(x_val, 
                                        data_path = acne_dataset + "JPEGImages", 
                                        transform=transform, 
                                        training=True)
                                        
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [14]:
testset = Classification_Dataset(test_df,
                                data_path = acne_dataset + "JPEGImages",
                                transform=test_transform,
                                training=True)

test_loader = torch.utils.data.DataLoader(testset, 
                                        batch_size=1,
                                        shuffle=False)

In [15]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [16]:
model = MyNet().cuda()

d:\Academics\Sem8\DMLAB\skin\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Academics\Sem8\DMLAB\skin\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=4, factor=0.5
    )

device = torch.device("cuda")

In [ ]:
from tqdm import tqdm
for param in model.parameters():
    param.requires_grad = True

model = model.to(device)
num_epoch = 10
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        device,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "saved_models/acne-severity" +  f"best_{i}.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

In [19]:
test_model = torch.load("saved_models/acne-severity/best_6.pt")

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [21]:

test_model = test_model.to(device)

In [22]:
def test_result(model, test_loader, device,name='no_tta_prob.npy'):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    aprobs = []
    labels = []
    with torch.no_grad():
        for data,target in test_loader:
            # move-tensors-to-GPU
            data = data.to(device)
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            prob = nn.Softmax(dim=1)
            # applying Softmax to results
            probs = prob(output)
            aprobs.append(probs.cpu())
            labels.append(target.cpu().numpy())
            preds.extend(torch.argmax(probs, axis=1).tolist())
    aprobs = np.array(aprobs)
    # np.save(name,aprobs)
    return preds, np.array(labels)

In [23]:
preds,labels =test_result(test_model, test_loader, device)
from sklearn.metrics import classification_report as rp
print(rp(labels,preds))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85       103
           1       0.80      0.76      0.78       127
           2       0.55      0.47      0.51        36
           3       0.83      0.73      0.78        26

    accuracy                           0.77       292
   macro avg       0.74      0.72      0.73       292
weighted avg       0.77      0.77      0.77       292

